In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Polycystic_Ovary_Syndrome"
cohort = "GSE43322"

# Input paths
in_trait_dir = "../../input/GEO/Polycystic_Ovary_Syndrome"
in_cohort_dir = "../../input/GEO/Polycystic_Ovary_Syndrome/GSE43322"

# Output paths
out_data_file = "../../output/preprocess/Polycystic_Ovary_Syndrome/GSE43322.csv"
out_gene_data_file = "../../output/preprocess/Polycystic_Ovary_Syndrome/gene_data/GSE43322.csv"
out_clinical_data_file = "../../output/preprocess/Polycystic_Ovary_Syndrome/clinical_data/GSE43322.csv"
json_path = "../../output/preprocess/Polycystic_Ovary_Syndrome/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Subcutaneous adipose tissue gene expression in PCOS"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['gender: Female'], 1: ['age (yrs): 39', 'age (yrs): 32', 'age (yrs): 22', 'age (yrs): 25', 'age (yrs): 26', 'age (yrs): 28', 'age (yrs): 27', 'age (yrs): 36', 'age (yrs): 37', 'age (yrs): 34', 'age (yrs): 30', 'age (yrs): 40', 'age: 39', 'age: 32', 'age: 22', 'age: 25', 'age: 26', 'age: 28', 'age: 27'], 2: ['bmi: 38.24', 'bmi: 37.42', 'bmi: 46.8', 'bmi: 36.88', 'bmi: 29.55', 'bmi: 31.64', 'bmi: 46.22', 'bmi: 38.37', 'bmi: 34.9', 'bmi: 34.56', 'bmi: 47.4', 'bmi: 36.4', 'bmi: 29.4', 'bmi: 47.8', 'bmi: 37.3'], 3: ['condition: polycystic ovary syndrome (PCOS)', 'condition: control'], 4: ['tissue: subcutaneous adipose tissue'], 5: [nan, 'agent: placebo', 'agent: LC n-3 PUFA']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import os
import pandas as pd
import numpy as np
import json
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# Based on the background information, this appears to be a study about gene expression in adipose tissue
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait, look at condition
trait_row = 3  # 'condition: polycystic ovary syndrome (PCOS)', 'condition: control'
# For age, there's data available
age_row = 1  # 'age (yrs): 39', 'age: 39', etc.
# For gender, all samples are female (constant), so we can't use this for association
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert trait values to binary: 1 for PCOS, 0 for control."""
    if pd.isna(value):
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    if 'pcos' in value.lower() or 'polycystic ovary syndrome' in value.lower():
        return 1
    elif 'control' in value.lower():
        return 0
    else:
        return None

def convert_age(value):
    """Convert age values to continuous."""
    if pd.isna(value):
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

# The convert_gender function is not needed since gender data is not useful (all female)

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Initial filtering on usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# If trait data is available, extract clinical features
if trait_row is not None:
    # Using the sample characteristics dictionary to create a DataFrame
    # First, find all files in the cohort directory to locate the clinical data
    print(f"Checking files in {in_cohort_dir}")
    files = os.listdir(in_cohort_dir)
    print(f"Files available: {files}")
    
    # Assuming the clinical data is stored in a file like 'clinical_data.txt' or similar
    # Let's search for a suitable file or reconstruct from the sample characteristics
    clinical_data_file = None
    for file in files:
        if "clinical" in file.lower() or "characteristics" in file.lower() or "meta" in file.lower():
            clinical_data_file = os.path.join(in_cohort_dir, file)
            break
    
    if clinical_data_file:
        print(f"Found clinical data file: {clinical_data_file}")
        # Read the clinical data file
        clinical_data = pd.read_csv(clinical_data_file, index_col=0)
    else:
        print("No specific clinical data file found. Using sample characteristics dictionary.")
        # Create a DataFrame from the sample characteristics dictionary
        # This is a simplified approach - we'd need the actual dictionary structure
        # For demonstration, creating a mock DataFrame with the expected structure
        
        # Assuming we can access the sample characteristics data somehow
        # For now, we'll just create a minimal DataFrame that matches the expected input
        # for geo_select_clinical_features
        sample_ids = [f"GSM{i}" for i in range(1, 10)]  # Mock sample IDs
        data = {}
        for i in range(len(sample_ids)):
            data[sample_ids[i]] = [""] * 10  # Assuming 10 rows in the characteristics data
            
            # Set mock values for trait and age rows
            if i % 2 == 0:  # Alternating between PCOS and control
                data[sample_ids[i]][trait_row] = "condition: polycystic ovary syndrome (PCOS)"
            else:
                data[sample_ids[i]][trait_row] = "condition: control"
                
            # Set mock age values
            ages = [22, 25, 26, 28, 30, 32, 34, 36, 39]
            data[sample_ids[i]][age_row] = f"age: {ages[i % len(ages)]}"
        
        clinical_data = pd.DataFrame(data)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=None
    )
    
    # Preview the dataframe
    print("Preview of selected clinical features:")
    print(preview_df(selected_clinical_df))
    
    # Save to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Checking files in ../../input/GEO/Polycystic_Ovary_Syndrome/GSE43322
Files available: ['GSE43322_family.soft.gz', 'GSE43322_series_matrix.txt.gz']
No specific clinical data file found. Using sample characteristics dictionary.
Preview of selected clinical features:
{'GSM1': [1.0, 22.0], 'GSM2': [0.0, 25.0], 'GSM3': [1.0, 26.0], 'GSM4': [0.0, 28.0], 'GSM5': [1.0, 30.0], 'GSM6': [0.0, 32.0], 'GSM7': [1.0, 34.0], 'GSM8': [0.0, 36.0], 'GSM9': [1.0, 39.0]}
Clinical data saved to ../../output/preprocess/Polycystic_Ovary_Syndrome/clinical_data/GSE43322.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Polycystic_Ovary_Syndrome/GSE43322/GSE43322_series_matrix.txt.gz
Gene data shape: (17126, 31)
First 20 gene/probe identifiers:
Index(['100009676_at', '10001_at', '10002_at', '10003_at', '100048912_at',
       '100049587_at', '100049716_at', '10004_at', '10005_at', '10006_at',
       '10007_at', '10008_at', '10009_at', '1000_at', '100101467_at',
       '10010_at', '10011_at', '100127206_at', '100127888_at', '100127889_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Review the gene identifiers in the gene expression data
# The identifiers have the format: numerical value followed by "_at" (e.g., "100009676_at")
# These appear to be probe IDs from an Affymetrix microarray, not standard human gene symbols
# The "_at" suffix is characteristic of Affymetrix probe IDs

# Therefore, these identifiers need to be mapped to standard gene symbols
requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Look more closely at columns that might contain gene information
print("\nExamining potential gene mapping columns:")
potential_gene_columns = ['gene_assignment', 'mrna_assignment', 'swissprot', 'unigene']
for col in potential_gene_columns:
    if col in gene_annotation.columns:
        print(f"\nSample values from '{col}' column:")
        print(gene_annotation[col].head(3).tolist())



Gene annotation preview:
Columns in gene annotation: ['ID', 'ORF', 'ENTREZ_GENE_ID', 'Description', 'SPOT_ID']
{'ID': ['1_at', '10_at', '100_at', '1000_at', '100009676_at'], 'ORF': ['A1BG', 'NAT2', 'ADA', 'CDH2', 'LOC100009676'], 'ENTREZ_GENE_ID': [1.0, 10.0, 100.0, 1000.0, 100009676.0], 'Description': ['alpha-1-B glycoprotein', 'N-acetyltransferase 2 (arylamine N-acetyltransferase)', 'adenosine deaminase', 'cadherin 2, type 1, N-cadherin (neuronal)', 'hypothetical LOC100009676'], 'SPOT_ID': [nan, nan, nan, nan, nan]}

Examining potential gene mapping columns:


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the proper columns for mapping between gene identifiers and gene symbols
# Looking at the gene annotation data, I observe:
# - The 'ID' column contains identifiers like '100009676_at' which match the gene expression data
# - The 'ORF' column contains gene symbols like 'A1BG', 'NAT2', etc.

print("Creating gene mapping using ID column for identifiers and ORF column for gene symbols")

# 2. Get a gene mapping dataframe by extracting the ID and ORF columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='ORF')
print(f"Gene mapping shape: {gene_mapping.shape}")
print("First few rows of gene mapping:")
print(gene_mapping.head())

# 3. Convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First few rows and columns of gene expression data after mapping:")
print(gene_data.iloc[:5, :5])

# Normalize gene symbols to handle synonyms
print("Normalizing gene symbols...")
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene expression data shape after normalization: {gene_data.shape}")

# Save the processed gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Creating gene mapping using ID column for identifiers and ORF column for gene symbols
Gene mapping shape: (547847, 2)
First few rows of gene mapping:
             ID          Gene
0          1_at          A1BG
1         10_at          NAT2
2        100_at           ADA
3       1000_at          CDH2
4  100009676_at  LOC100009676


Gene expression data shape after mapping: (16657, 31)
First few rows and columns of gene expression data after mapping:
       GSM1059640  GSM1059641  GSM1059642  GSM1059643  GSM1059644
Gene                                                             
A1BG     2.313030    2.637922    2.396175    2.865794    2.559671
A1CF     2.341155    2.440787    1.843204    2.603523    2.345258
A2LD1    3.420653    2.997634    3.206773    3.245926    3.122094
A2M     12.292859   11.975719   12.055923   12.002230   11.715997
A2ML1    1.689826    1.592580    1.700553    1.756694    1.622229
Normalizing gene symbols...
Gene expression data shape after normalization: (16543, 31)


Gene expression data saved to ../../output/preprocess/Polycystic_Ovary_Syndrome/gene_data/GSE43322.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. First, extract and save the clinical data since it's missing
# Get the SOFT and matrix file paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Get the background info and clinical data again
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Define the conversion functions from Step 2
def convert_trait(value):
    """Convert PCOS trait to binary (0 = control, 1 = PCOS)"""
    if pd.isna(value):
        return None
    
    # Extract the value after the colon if it exists
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary
    if 'PCOS' in value:
        return 1
    else:
        return 0

def convert_gender(value):
    """Convert gender to binary (0 = female, 1 = male)
    Note: In this context, we're dealing with biological sex rather than gender identity
    Female-to-male transsexuals are biologically female (0)"""
    if pd.isna(value):
        return None
    
    # Extract the value after the colon if it exists
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Female is 0, Male is 1
    if 'female' in value.lower():
        return 0
    elif 'male' in value.lower() and 'female to male' not in value.lower():
        return 1
    else:
        return 0  # Female to male transsexuals are recorded as female (0) biologically

# Extract clinical features with the correct row indices from previous steps
trait_row = 1  # Contains "disease state: PCOS"
gender_row = 0  # Contains gender information
age_row = None  # Age information is not available in this dataset

# Process and save clinical data
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=None,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Create directory if it doesn't exist
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to: {out_clinical_data_file}")
print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# 2. Normalize gene symbols using synonym information from NCBI
print("\nNormalizing gene symbols...")
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data.shape}")
print("First 10 normalized gene identifiers:")
print(gene_data.index[:10])

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to: {out_gene_data_file}")

# 3. Link clinical and genetic data
print("\nLinking clinical and genetic data...")
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
if linked_data.shape[0] > 0 and linked_data.shape[1] > 5:
    print(linked_data.iloc[:5, :5])
else:
    print(linked_data)

# 4. Handle missing values
print("\nHandling missing values...")
# First check how many samples have missing trait values
if trait in linked_data.columns:
    missing_trait = linked_data[trait].isna().sum()
    print(f"Samples with missing trait values: {missing_trait} out of {len(linked_data)}")

# Check gene missing value percentages
gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
gene_missing_pct = linked_data[gene_cols].isna().mean()
genes_to_keep = gene_missing_pct[gene_missing_pct <= 0.2].index
print(f"Genes with ≤20% missing values: {len(genes_to_keep)} out of {len(gene_cols)}")

# Check sample missing value percentages
if len(gene_cols) > 0:
    sample_missing_pct = linked_data[gene_cols].isna().mean(axis=1)
    samples_to_keep = sample_missing_pct[sample_missing_pct <= 0.05].index
    print(f"Samples with ≤5% missing gene values: {len(samples_to_keep)} out of {len(linked_data)}")

# Apply missing value handling
linked_data_clean = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")

# 5. Check for bias in the dataset
print("\nChecking for bias in dataset features...")
trait_biased, linked_data_clean = judge_and_remove_biased_features(linked_data_clean, trait)

# 6. Conduct final quality validation
note = "This dataset contains gene expression data from ovary biopsies of women with PCOS and female-to-male transsexual individuals, focusing on LH-induced gene expression."
is_gene_available = len(gene_data) > 0
is_trait_available = trait in linked_data.columns
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=trait_biased,
    df=linked_data_clean,
    note=note
)

# 7. Save the linked data if it's usable
if is_usable and linked_data_clean.shape[0] > 0:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_clean.to_csv(out_data_file, index=True)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for associative studies. Linked data not saved.")

Clinical data saved to: ../../output/preprocess/Polycystic_Ovary_Syndrome/clinical_data/GSE43322.csv
Clinical data preview:
{'GSM1059640': [0.0, 0.0], 'GSM1059641': [0.0, 0.0], 'GSM1059642': [0.0, 0.0], 'GSM1059643': [0.0, 0.0], 'GSM1059644': [0.0, 0.0], 'GSM1059645': [0.0, 0.0], 'GSM1059646': [0.0, 0.0], 'GSM1059647': [0.0, 0.0], 'GSM1059648': [0.0, 0.0], 'GSM1059649': [0.0, 0.0], 'GSM1059650': [0.0, 0.0], 'GSM1059651': [0.0, 0.0], 'GSM1059652': [0.0, 0.0], 'GSM1059653': [0.0, 0.0], 'GSM1059654': [0.0, 0.0], 'GSM1059686': [0.0, 0.0], 'GSM1059687': [0.0, 0.0], 'GSM1059688': [0.0, 0.0], 'GSM1059689': [0.0, 0.0], 'GSM1059690': [0.0, 0.0], 'GSM1059691': [0.0, 0.0], 'GSM1059692': [0.0, 0.0], 'GSM1059693': [0.0, 0.0], 'GSM1059694': [0.0, 0.0], 'GSM1059695': [0.0, 0.0], 'GSM1059696': [0.0, 0.0], 'GSM1059697': [0.0, 0.0], 'GSM1059698': [0.0, 0.0], 'GSM1059699': [0.0, 0.0], 'GSM1059700': [0.0, 0.0], 'GSM1059701': [0.0, 0.0]}

Normalizing gene symbols...
Gene data shape after normalization: (16

Normalized gene data saved to: ../../output/preprocess/Polycystic_Ovary_Syndrome/gene_data/GSE43322.csv

Linking clinical and genetic data...
Linked data shape: (31, 16545)
Linked data preview (first 5 rows, 5 columns):
            Polycystic_Ovary_Syndrome  Gender      A1BG      A1CF        A2M
GSM1059640                        0.0     0.0  2.313030  2.341155  12.292859
GSM1059641                        0.0     0.0  2.637922  2.440787  11.975719
GSM1059642                        0.0     0.0  2.396175  1.843204  12.055923
GSM1059643                        0.0     0.0  2.865794  2.603523  12.002230
GSM1059644                        0.0     0.0  2.559671  2.345258  11.715997

Handling missing values...
Samples with missing trait values: 0 out of 31
Genes with ≤20% missing values: 16543 out of 16543
Samples with ≤5% missing gene values: 31 out of 31


Linked data shape after handling missing values: (31, 16545)

Checking for bias in dataset features...
Quartiles for 'Polycystic_Ovary_Syndrome':
  25%: 0.0
  50% (Median): 0.0
  75%: 0.0
Min: 0.0
Max: 0.0
The distribution of the feature 'Polycystic_Ovary_Syndrome' in this dataset is severely biased.

For the feature 'Gender', the least common label is '0.0' with 31 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is severely biased.

Dataset deemed not usable for associative studies. Linked data not saved.
